In [2]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import numpy as np
import pandas as pd
import yfinance as yf

from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

import wavy

In [4]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].Symbol

In [4]:
msft = yf.Ticker("MSFT")

# get stock info
msft.info

# get historical market data from 2005 on
hist = msft.history(period="max", start="2005-01-01")

KeyboardInterrupt: 

In [6]:
hist.head(10)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2005-01-03,19.040897,19.147470,18.934325,18.998268,65002900,0.0,0
2005-01-04,19.090624,19.254034,18.941422,19.069309,109442100,0.0,0
2005-01-05,19.069312,19.254038,19.012474,19.026684,72463500,0.0,0
2005-01-06,19.076419,19.225620,18.927218,19.005371,76890500,0.0,0
2005-01-07,19.055110,19.104843,18.913014,18.948538,68723300,0.0,0
2005-01-10,18.898802,19.083527,18.856173,19.040897,70376600,0.0,0
2005-01-11,18.962745,19.055107,18.905906,18.991163,64712000,0.0,0
2005-01-12,19.019579,19.076417,18.913007,19.026684,72940600,0.0,0
2005-01-13,18.955630,19.040887,18.586180,18.664333,89861600,0.0,0


In [7]:
hist[['Open', 'High', 'Low', 'Close']].plot()

In [8]:
hist.Dividends.plot()

In [9]:
hist['Stock Splits'].plot()

In [10]:
hist = hist[['Open', 'High', 'Low', 'Close']]

In [11]:
hist = hist.pct_change()

In [12]:
hist.Close.plot()

In [13]:
print(len(hist) - len(hist.dropna()))
hist.dropna(inplace=True)

1


In [14]:
hist.High.hist(bins=200)

In [15]:
x, y = wavy.create_panels(hist, lookback=10, horizon=1)

In [16]:
y[0]

size                           1
timesteps                      1
start        2005-01-19 00:00:00
end          2005-01-19 00:00:00
Name: Panel, dtype: object


<Panel, size 1>

In [17]:
# predict only the highest price
y = y[['High']]

In [18]:
# try single col inputs first
x_high = x[['High']]

In [19]:
# constant = wavy.ConstantModel(x_high, y, model_type="regression")
# constant.fit()

In [29]:
baseline = wavy.BaselineModel(x_high, y, model_type="regression")
baseline.fit() # val_MAE: 0.0136

RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
# How to interpret that error? What does it mean comparing to the mean/std of the val set?
np.median(baseline.y_val.flatten())

In [ ]:
lr = wavy.LinearRegression(x_high, y)
lr.fit(epochs=20)

In [ ]:
y.test[0]

In [ ]:
y.test[1]

In [ ]:
baseline.x_test.shape

In [ ]:
from copy import deepcopy

def update(panel, other):
    panel = deepcopy(panel)

    for i, j in zip(panel, other):
        i.iloc[:,:] = j
    return panel

In [ ]:
y.test.shape

In [ ]:
a = update(y.test[1:], baseline.predict())

In [ ]:
a[0]

In [ ]:
y.test[1]

In [ ]:
copy(y.test)

In [ ]:
b = baseline.predict()

In [ ]:
b[0]

In [ ]:
dense = wavy.DenseModel(x_high, y, model_type="regression")
dense.fit(epochs=20)

In [ ]:
conv = wavy.ConvModel(x_high, y, model_type="regression")
conv.fit(epochs=20)

# using a cannon ball to kill an ant!

In [ ]:
dummy = wavy.ShallowModel(x_high, y, model=DummyRegressor, metrics=[mean_absolute_error], strategy="mean")
dummy.fit()

In [ ]:
rf = wavy.ShallowModel(x, y, model=RandomForestRegressor, metrics=[mean_absolute_error], n_estimators=50)
rf.fit()

In [ ]:
dense = wavy.DenseModel(x, y, model_type="regression")
dense.fit(epochs=20)

In [ ]:
# Try the same models but with all input columns
# TODO: Use function to compute default results per model

# lr = wavy.LinearRegression(x, y)
# lr.fit(epochs=20)